In [3]:
import json
from pprint import pprint
import random

In [4]:
http_keys = ['httpHeaders', 'httpStatusCode', 'httpStatusMessage', 'httpVersion']
http_obs = []
with open('http_test', 'r') as f:
    for line in f:
        d = json.loads(line)
        http = {http_key: d[http_key] for http_key in http_keys}
        http['ipv4'] = d['tlsEndpoint']['ip']['ipv4']
        http['port'] = d['tlsEndpoint']['portNumber']
        http['protocol'] = d['tlsEndpoint']['portProtocol']
        http_obs.append(http)
pprint(http_obs[:1])

[{'httpHeaders': [{'name': 'Date', 'value': 'Sun, 19 Jan 2020 19:58:04 GMT'},
                  {'name': 'Server',
                   'value': 'Apache/2.4.39 (Amazon) OpenSSL/1.0.2k-fips'},
                  {'name': 'X-Amzn-Trace-Id',
                   'value': 'Root=1-5e24b4cc-323845ea1cc8eced463cb7a7;'},
                  {'name': 'Content-Type', 'value': 'text/html;charset=UTF-8'},
                  {'name': 'Content-Length', 'value': '6303'},
                  {'name': 'Set-Cookie',
                   'value': 'JSESSIONID=2BCA4477FD021C9D2A78BAF4598D1D8C; '
                            'Path=/; HttpOnly'},
                  {'name': 'Vary', 'value': 'Accept-Encoding'},
                  {'name': 'Connection', 'value': 'close'}],
  'httpStatusCode': '200',
  'httpStatusMessage': '',
  'httpVersion': '1.1',
  'ipv4': '3.133.76.2',
  'port': '80',
  'protocol': 'TCP'}]


In [6]:
rdp_keys = ['algorithms', 'levels', 'protocols']
rdp_obs = []
with open('rdp_test', 'r') as f:
    for line in f:
        d = json.loads(line)
        rdp = {rdp_key: d[rdp_key] for rdp_key in rdp_keys if rdp_key in d}
        rdp['ipv4'] = d['endpoint']['ip']['ipv4']
        rdp['port'] = d['endpoint']['portNumber']
        rdp['protocol'] = d['endpoint']['portProtocol']
        rdp_obs.append(rdp)
pprint(rdp_obs[:1])

[{'algorithms': {'data': [{'key': '56-bit-RC4', 'value': True},
                          {'key': 'FIPS-140-1', 'value': False},
                          {'key': '128-bit-RC4', 'value': True},
                          {'key': '40-bit-RC4', 'value': True}]},
  'ipv4': '13.115.141.24',
  'levels': {'data': [{'key': 'Client Compatible', 'value': True}]},
  'port': '3389',
  'protocol': 'TCP',
  'protocols': {'data': [{'key': 'CredSSP', 'value': False},
                         {'key': 'SSL', 'value': True},
                         {'key': 'NativeRDP', 'value': True}]}}]


In [7]:
rand_ips = []
for _ in range(1500):
    ip = ".".join(map(str, (random.randint(0, 255) for _ in range(4))))
    rand_ips.append(ip)

In [8]:
for i in range(500):
    http_obs[i]['ipv4'] = rand_ips[i]
    rdp_obs[i]['ipv4'] = rand_ips[i]
for i in range(500, 1000):
    http_obs[i]['ipv4'] = rand_ips[i]
for i in range(500, 1000):
    rdp_obs[i]['ipv4'] = rand_ips[i+500]

In [9]:
obs = {}
for http in http_obs:
    obs[http['ipv4']] = {'http': http}
for rdp in rdp_obs:
    if rdp['ipv4'] in obs:
        obs[rdp['ipv4']]['rdp'] = rdp
    else:
        obs[rdp['ipv4']] = {'rdp': rdp}

In [10]:
matching_ips = []
for ip, o in obs.items():
    if 'rdp' in o:
        for d in o['rdp']['protocols']['data']:
            if d['key'] == 'CredSSP' and d['value'] == True:
                if 'http' in o:
                    for header in o['http']['httpHeaders']:
                        if header['name'] == 'Server' and header['value'] == 'nginx':
                            matching_ips.append(ip)
                            
ips_to_query = matching_ips + [rand_ips[501], rand_ips[1001], '1.1.1.1']
print(ips_to_query)

['192.31.68.255', '200.197.173.155', '63.161.232.91', '248.145.171.237', '109.239.245.111', '1.1.1.1']


In [11]:
with open('http_interview.json', 'w+') as f:
    for http in http_obs:
        f.write(json.dumps(http))
        f.write('\n')
with open('rdp_interview.json', 'w+') as f:
    for rdp in rdp_obs:
        f.write(json.dumps(rdp))
        f.write('\n')
with open('ips_to_query.txt', 'w+') as f:
    for ip in ips_to_query:
        f.write(ip)
        f.write('\n')